<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo

El presente notebook corresponde al desafío N°2 del curso de Procesamiento del Lenguaje Natural. Está basado fuertemente en el material presentado en clase, con algunas diferecias ya que se utilizará un corpus diferente. En resumen, la tarea consiste en:

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

In [80]:
# Se importan los módulos necesarios

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

import plotly.graph_objects as go
import plotly.express as px

### Datos

Se utilizará como dataset un libro de música llamado "Essentials of Music Theory: Elementary" del sitio *https://www.gutenberg.org*.

Con esta información se crearán los embeddings de palabras, cuya representación se basará en el contenido del libro. Se analizarán relaciones basadas en la semántica de los conceptos musicales.

In [81]:
# Se arma el dataset a partir del libro "Essentials of Music Theory: Elementary" de Project Gutenberg
# Se utiliza el salto de línea para separar las oraciones/documentos.
df = pd.read_csv('https://www.gutenberg.org/cache/epub/65500/pg65500.txt', sep='/n', header=None)

# Se muestran las primeras líneas para verificar la carga
df.head()

<ipython-input-81-6b11cf74f03b>:3: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,0
0,The Project Gutenberg eBook of Essentials of M...
1,This ebook is for the use of anyone anywhere i...
2,most other parts of the world at no cost and w...
3,"whatsoever. You may copy it, give it away or r..."
4,of the Project Gutenberg License included with...


In [82]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 1938


### 1 - Preprocesamiento

In [83]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones en una secuencia de palabras
for _, row in df.iterrows():
    if isinstance(row[0], str):
        sentence_tokens.append(text_to_word_sequence(row[0]))


In [84]:
# Se observan las transformaciones para las primeras dos líneas
sentence_tokens[:2]

[['the',
  'project',
  'gutenberg',
  'ebook',
  'of',
  'essentials',
  'of',
  'music',
  'theory',
  'elementary'],
 ['this',
  'ebook',
  'is',
  'for',
  'the',
  'use',
  'of',
  'anyone',
  'anywhere',
  'in',
  'the',
  'united',
  'states',
  'and']]

### 2 - Crear los vectores (word2vec)

Se utiliza el mismo *callback* presentado en clase, ya que proporciona información adecuada sobre el proceso de entrenamiento.

In [85]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

En la siguiente celda se crea el modelo de vectorización Word2Vec utilizando la arquitectura Skipgram. También se hicieron pruebas con CBOW pero no fue tan efectivo capturando relaciones semánticas en conceptos musicales más complejos.

Se generaron embeddings de tamaño 50 ya que el vocabulario utilizado por el corpus no es tan amplio. La ventana utilizada fue de 3 palabras alrededor de la palabra objetivo dado que se obtuvieron mejores resultados con este valor.

In [86]:
# Se crea modelo vectorizador con Skipgram
w2v_model = Word2Vec(min_count=1,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=3,       # cant de palabras antes y desp del objetivo
                     vector_size=50, # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=multiprocessing.cpu_count(),
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [87]:
# Se obtiene el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [88]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1938


In [89]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 2037


### 3 - Entrenar embeddings

In [90]:
# Se entrena el modelo (utilizando el callback definido)
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 161762.0625
Loss after epoch 1: 92761.6875
Loss after epoch 2: 84672.25
Loss after epoch 3: 79323.625
Loss after epoch 4: 78844.875
Loss after epoch 5: 77036.0625
Loss after epoch 6: 75518.875
Loss after epoch 7: 73118.25
Loss after epoch 8: 72045.6875
Loss after epoch 9: 69975.9375
Loss after epoch 10: 70013.375
Loss after epoch 11: 68522.875
Loss after epoch 12: 65350.9375
Loss after epoch 13: 62027.625
Loss after epoch 14: 60640.375
Loss after epoch 15: 60806.5
Loss after epoch 16: 61396.75
Loss after epoch 17: 58735.75
Loss after epoch 18: 60521.0
Loss after epoch 19: 59824.375


(235210, 326340)

### 4 - Ensayar

En las siguientes celdas se analizarán las relaciones entre palabras en el espacio de embeddings.

In [91]:
# Se obtienen las palabras que más se relacionan con el concepto de sostenido (sharp)
w2v_model.wv.most_similar(positive=["sharp"], topn=10)

[('flat', 0.969540536403656),
 ('sharped', 0.93711918592453),
 ('added', 0.9298794269561768),
 ('single', 0.9102452397346497),
 ('cancel', 0.9093285202980042),
 ('signatures', 0.8978573679924011),
 ('signature', 0.8899998068809509),
 ('lowers', 0.8886432647705078),
 ('remains', 0.8864120244979858),
 ('order', 0.8831487894058228)]

En este caso la similitud con bemol (*flat*) es bastante predecible, ya que ambas son alteraciones que se pueden hacer a una nota musical. *Sharp* aumenta la frecuencia en medio tono mientras que *flat* la disminuye también en medio tono.

In [92]:
# Se obtienen palabras que menos se relacionan con music
w2v_model.wv.most_similar(negative=["music"], topn=10)

[('edition', 0.4294172525405884),
 ('tyndall', 0.2931167483329773),
 ('pole', 0.21923333406448364),
 ('wings', 0.1508379429578781),
 ('reference', -0.03322002291679382),
 ('electronic', -0.24459458887577057),
 ('states', -0.26868122816085815),
 ('not', -0.26922836899757385),
 ('step', -0.2795872986316681),
 ('more', -0.28001099824905396)]

En este caso, se observan palabras que claramente no tienen ninguna relación con el concepto de música. Tal vez *electronic* sí podría estar relacionada, pero al ser un texto que establece conceptos musicales básicos, no se menciona en ningún la palabra *electronic* como un concepto relacionado a la música.

In [93]:
# Se obtienen palabras similares a chord (acorde)
w2v_model.wv.most_similar(positive=["chord"], topn=10)

[('dominant', 0.9170248508453369),
 ('inversion', 0.8822497129440308),
 ('fundamental', 0.8799612522125244),
 ('figured', 0.8774280548095703),
 ('first', 0.8692854046821594),
 ('subtonic', 0.8688098788261414),
 ('because', 0.8634206652641296),
 ('seventh', 0.8602502346038818),
 ('sixth', 0.8560895919799805),
 ('doubled', 0.8466036319732666)]

Aquí se puede observar que las palabras más similares corresponden a elementos que componen un acorde (*dominant*, *fundamental*) o alteraciones a los mismos (*inversion*). La palabra *figured* aparece entre las primeras ya que generalmente en el libro se utiliza generalmente para indicar cómo esta formado un acorde.

In [94]:
# Se obtienen las palabras que más se relacionan con interval
w2v_model.wv.most_similar(positive=["interval"], topn=5)

[('octave', 0.8790116906166077),
 ('step', 0.8788273334503174),
 ('situated', 0.8653691411018372),
 ('being', 0.8652088642120361),
 ('accidental', 0.8625380396842957)]

Las palabras que más se relacionan con *interval* tienen bastante sentido. *octave* se refiere al intervalo de octava, mientras que *step* se utiliza muchas veces como sinónimo de intervalo. El resto de las palabras que siguen no están tan relacionadas con *interval* musicalmente, pero seguramente en el texto sean utilizadas para definirlos y explicarlos.

Al momento de analizar las analogías se encontró que al ser un texto tan pequeño y con la información dispuesta en diversas notaciones, el aprendizaje de las similaridades no fue tan efectivo como se esperaba. Por ejemplo, para nombrar un intervalo musical de séptima se usa tanto la palabra "7th" como "seventh". En un libro más extenso tal vez se hubiese podido capturar estas relaciones de forma más efectiva, de la mano con embeddings de mayor tamaño y un modelo más potente.

De todas formas se pudieron capturar algunas analogías interesantes. Una de ellas es ver qué sucede con los sostenidos y bemoles. Un sostenido eleva una nota medio tono más alto, mientra que el bemol hace lo contrario, baja medio tono. Por lo tanto, se analiza la analogía "sharp" - "high" + "low", que básicamente le quita lo "alto" al sostenido y le agrega lo "bajo".

In [95]:
result = w2v_model.wv.most_similar(positive=['sharp','low'], negative=['high'], topn=1)
print(result)

[('flat', 0.9462301135063171)]


El resultado *flat* (bemol) es coherente con la operación realizada.

En un piano, las teclas blancas son las naturales mientras que las negras son la alteradas (sostenidos y bemoles). Si se realiza la analogía 'white' + 'key' - 'natural' se obtiene 'black', lo cual nuevamente tiene sentido. De todas formas, en este caso la similaridad no fue tan fuerte como en el caso anterior.

In [96]:
result = w2v_model.wv.most_similar(positive=['white','key'], negative=['natural'], topn=1)
print(result)

[('black', 0.8111461400985718)]


### 5 - Visualizar agrupación de vectores

En las siguientes celdas se realizarán representaciones en 2D y 3D de los embeddings de palabras, de forma de analizar si las similitudes también pueden ser detectadas visualmente.

In [97]:
def reduce_dimensions(model, num_dimensions=2):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [98]:
# Graficar los embedddings en 2D
vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

En la representación se puede observar como las palabras con significados similares se agrupan en clusters. Por ejemplo, los conceptos relacionados con acordes y su formación aparecen cercanos (ej. "chord", "triad", "tonic") así como también los intervalos ("2nd", "3rd", etc.).

Otras palabras ya no tan relacionadas a la música se pueden ver pero algo más dispersas, ya que son utilizadas en muchos contextos diferentes y no en un cluster particular.

In [99]:
# Se grafican los embeddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab")

En esta representación tridimensional se aprecia una mayor dispersión de los puntos y tal vez resulte un poco más difícil la visualización. De todas formas, es posible notar algunos grupos con significados similares que también se podían ver en dos dimensiones. Dos vectores cuya representación es muy cercana en este caso es "student" y "teacher" que aparecen a una distancia relativamente pequeña.

Claramente, también se puede observar que los conceptos que aparecían agrupados en la versión bidimensional, también lo están aquí por lo general.

A continuación se generan archivos tsv con los vectores y metadatos para poder visualizarlos en http://projector.tensorflow.org/.

In [100]:
# Se generan vectores y labels como tsv para visualización en http://projector.tensorflow.org/
vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

En la visualización de http://projector.tensorflow.org/ se observan con mayor claridad las agrupaciones de términos relacionados. Las palabras con significados similares se agrupan en clusters más definidos, lo cual facilita la detección de relaciones semánticas. Además, su interfaz permite fácilmente variar entre diferentes ténicas de reducción de dimensionalidad (ej. TCA, t-SNE, UMAP).

En el directorio de este trabajo se adjuntan los archivos *vectors.tsv* y *labels.tsv* generados para poder desplegarlos en dicha plataforma.

### Conclusiones generales

En este trabajo se creó un modelo Word2Vec con arquitectura Skipgram que capturas algunas relaciones semánticas entre palabras basado en un libro de teoría musical. Se identificaron algunas analogías interesantes, pero también se intentaron otras más complejas que no tuvieron resultados coherentes. Esto puede deberse a que el tamaño del corpus es relativamente pequeño y que además los conceptos musicales son explicados de manera muy básica.

Seguramente se obtengan mejores resultados con un corpus de mayor tamaño que permita generar vectorizaciones con embeddings de mayor dimensionalidad, y así capturar relaciones semánticas más complejas.